In [24]:
!pip install transformers
!pip install nltk

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [29]:
import re
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [132]:
testString = """Beneath the amber-hued canopy, crystalline dewdrops adorned the emerald foliage like scattered diamonds. An ethereal mist lingered, curling around ancient oaks, their gnarled branches whispering secrets of forgotten epochs. Far in the distance, a brook meandered, its murmurs harmonizing with the symphony of chirping finches and rustling leaves. Each fleeting moment sparkled with the ephemeral beauty of dawn, a prelude to the day’s untold possibilities."""

In [36]:
def read_sentences(input):
    sentences = sent_tokenize(input)
    return sentences

In [162]:
def word_length(input):
  onetoFour = 0
  fivetoEight = 0
  nineMore = 0
  outputPrompt = ""
  vocabulary = {}
  for sentence in input:
    sentence = sentence.replace(",", " ")
    sentence = sentence.replace(";", " ")
    sentence = sentence.replace(".", " ")
    sentence = sentence.replace("  ", " ")
    words = sentence.split(" ")
    for word in words:
      if word in vocabulary:
        vocabulary[word] += 1
      else:
        vocabulary[word] = 1
      if len(word) < 5:
        onetoFour += 1
      elif len(word) < 9:
        fivetoEight += 1
      else:
        nineMore += 1
  total = onetoFour + fivetoEight + nineMore
  shortPercent = 100 * onetoFour / total
  HapaxLegomena = 0
  for word in vocabulary:
    if vocabulary[word] == 1:
      HapaxLegomena += 1
  HapaxPercent = 100 * HapaxLegomena / len(vocabulary)
  DisLegomena = 0
  for word in vocabulary:
    if vocabulary[word] == 2:
      DisLegomena += 1
  DisPercent = 100 * DisLegomena / len(vocabulary)
  SimpsonIndex = 0
  for word in vocabulary:
    SimpsonIndex += (vocabulary[word] ** 2)
  SimpsonIndex = SimpsonIndex / (len(vocabulary) ** 2)
  if shortPercent <= 45:
    outputPrompt += "The writing is using less short words, with about " + str(round(shortPercent)) +"% of words that only have a length of 1 to 4."
  elif shortPercent >= 65:
    outputPrompt += "The writing is using more short words, with about " + str(round(shortPercent)) +"% of words that only have a length of 1 to 4."
  longPercent = 100 * nineMore / total
  if longPercent >= 12:
    outputPrompt += "The writing is using more long words, with about " + str(round(longPercent)) +"% of words that have a length of 9 or more."
  elif longPercent <= 4:
    outputPrompt += "The writing is using less long words, with about " + str(round(longPercent)) +"% of words that have a length of 9 or more."
  if HapaxPercent >= 10:
    outputPrompt += "The writing is using more Hapax Legomena, with about " + str(round(HapaxPercent)) +"% of words that only appear once."
  elif HapaxPercent <= 5:
    outputPrompt += "The writing is using less Hapax Legomena, with about " + str(round(HapaxPercent)) +"% of words that only appear once."
  if DisPercent >= 7:
    outputPrompt += "The writing is using more Dislegomena, with about " + str(round(DisPercent)) +"% of words that only appear twice."
  elif DisPercent <= 3:
    outputPrompt += "The writing is using less Dislegomena, with about " + str(round(DisPercent)) +"% of words that only appear twice."
  if SimpsonIndex < 0.05:
    outputPrompt += "The writing have a highly diverse vocabulary with minimal repetition, with a Simpson's Index of " + str(round(SimpsonIndex, 2)) + "."
  elif SimpsonIndex > 0.15:
    outputPrompt += "The writing have a poor diverse vocabulary with significant repetition, with a Simpson's Index of " + str(round(SimpsonIndex, 2)) + "."
  return outputPrompt

In [130]:
def emotion(input):
  outputPrompt = ""
  emo = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'neutral': 0, 'sadness': 0, 'surprise': 0}
  for sentence in input:
    emotions = classifier(sentence)
    emotions = emotions[0]
    highestEmotions = ""
    highestScore = 0
    outputPrompt = ""
    for emotion in emotions:
      if (emotion['score']) > highestScore:
        highestScore = emotion['score']
        highestEmotions = emotion['label']
    emo[highestEmotions] += 1
  highestEmotions = ""
  highestScore = 0
  for emotion in emo:
    if (emo[emotion]) > highestScore:
      highestScore = emo[emotion]
      highestEmotions = emotion
  outputPrompt += "In general, this writing have a mood of " + highestEmotions + "."
  return outputPrompt

In [163]:
sentences = read_sentences(testString)
print(word_length(sentences))
print(emotion(sentences))

The writing is using less short words, with about 41% of words that only have a length of 1 to 4.The writing is using more long words, with about 13% of words that have a length of 9 or more.The writing is using more Hapax Legomena, with about 91% of words that only appear once.The writing have a highly diverse vocabulary with minimal repetition, with a Simpson's Index of 0.04.
In general, this writing have a mood of neutral.
